In [37]:
import os
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
from google.cloud import bigquery
import json

from google.cloud import storage


PROJECT_ID = "data-management-project-452400"
BUCKET_NAME = "data-mgmt-bucket"

# BigQuery details
BQ_DATASET_NAME = "data_mgmt_project"
BQ_PRODUCT_POPULARITY_TABLE_NAME = "top10_product_popularity"
BQ_BRAND_POPULARITY_TABLE_NAME = "top10_brand_popularity"
BQ_sub_category_POPULARITY_TABLE_NAME = "top10_sub_category_popularity"
BQ_gender_season_POPULARITY_TABLE_NAME = "top_10_products_by_season_gender"


POP_WEIGHTS_PATH = 'gs://data-mgmt-bucket/feature_importance.json'
# Set up Google Cloud authentication (Ensure you've set up your service account JSON)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = 'data-management-project-452400-74094d20e7ee.json'

In [2]:
def load_to_bq(client, df, table_path):
    job = client.load_table_from_dataframe(df, table_path, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))
    return job.result()

In [3]:
# ============================
# 1️⃣  CONNECT TO BIGQUERY
# ============================

# Initialize BigQuery Client
client = bigquery.Client()

# Define your BigQuery table
bq_table = "data-management-project-452400.data_mgmt_project.brands_data"

# Query BigQuery to get the data
query = f"""
SELECT 
    *
FROM `{bq_table}`
"""

# Load Data into Pandas DataFrame
df = client.query(query).to_dataframe()

c:\Users\utkar\.conda\envs\PyCaret\lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [7]:
df

,product_id,masterCategory,subCategory,articleType,season,click_count,item_detail_count,add_to_cart_count,total_dwell_time,homepage_count,purchases,productDisplayName,brands
0,1163,Apparel,Topwear,Tshirts,Summer,0,1,23,331219,12,39,Nike Sahara Team India Fanwear Round Neck Jersey,Nike
1,1164,Apparel,Topwear,Tshirts,Winter,2,3,20,602350,12,29,Nike Men Blue T20 Indian Cricket Jersey,Nike
2,1165,Apparel,Topwear,Tshirts,Summer,0,0,26,605988,15,31,Nike Mean Team India Cricket Jersey,Nike
3,1525,Accessories,Bags,Backpacks,Fall,19,6,73,2884996,23,47,Puma Deck Navy Blue Backpack,Puma
4,1526,Accessories,Bags,Backpacks,Fall,18,14,80,2462089,22,31,Puma Big Cat Backpack Black,Puma
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44441,59995,Apparel,Dress,Dresses,Fall,17,10,66,1788317,20,26,Avirate Black Dress,Avirate
44442,59996,Apparel,Dress,Dresses,Summer,9,1,77,1954794,21,29,Avirate Black & Purple Dress,Avirate
44443,59998,Footwear,Shoes,Heels,Winter,16,5,68,2254185,25,34,Catwalk Women Multi Coloured Heels,Catwalk
44444,59999,Footwear,Shoes,Heels,Winter,15,13,73,1884894,23,31,Catwalk Women Bronze Wedges,Catwalk


In [8]:
# Read in the weights for product popularity computation from GCS
bucket_name = BUCKET_NAME # Replace with actual bucket name
file_name = "feature_importance.json"  # JSON file stored in GCS

# Initialize GCS client
client = storage.Client()

# Get the GCS bucket and file
bucket = client.bucket(bucket_name)
blob = bucket.blob(file_name)

# Download JSON file as text and load it as a dictionary
json_data = json.loads(blob.download_as_text())

# Convert to Pandas Series instead of DataFrame
weights = pd.Series(json_data)

In [9]:
weights

click_count          0.05
item_detail_count    0.06
add_to_cart_count    0.75
total_dwell_time     0.05
homepage_count       0.09
purchases            1.10
dtype: float64

In [10]:
df["popularity_score"] = (df["click_count"] * weights["click_count"] +
    df["item_detail_count"] * weights["item_detail_count"] +
    df["add_to_cart_count"] * weights["add_to_cart_count"] +
    df["total_dwell_time"] * weights["total_dwell_time"] +
    df["homepage_count"] * weights["homepage_count"]+
    df["purchases"] * weights["purchases"])/6

In [11]:
# Min Max Normalize the popularity score
df["popularity_score"] = (df["popularity_score"] - df["popularity_score"].min()) / (df["popularity_score"].max() - df["popularity_score"].min())
df

,product_id,masterCategory,subCategory,articleType,season,click_count,item_detail_count,add_to_cart_count,total_dwell_time,homepage_count,purchases,productDisplayName,brands,popularity_score
0,1163,Apparel,Topwear,Tshirts,Summer,0,1,23,331219,12,39,Nike Sahara Team India Fanwear Round Neck Jersey,Nike,0.070414
1,1164,Apparel,Topwear,Tshirts,Winter,2,3,20,602350,12,29,Nike Men Blue T20 Indian Cricket Jersey,Nike,0.129746
2,1165,Apparel,Topwear,Tshirts,Summer,0,0,26,605988,15,31,Nike Mean Team India Cricket Jersey,Nike,0.130573
3,1525,Accessories,Bags,Backpacks,Fall,19,6,73,2884996,23,47,Puma Deck Navy Blue Backpack,Puma,0.630013
4,1526,Accessories,Bags,Backpacks,Fall,18,14,80,2462089,22,31,Puma Big Cat Backpack Black,Puma,0.537325
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44441,59995,Apparel,Dress,Dresses,Fall,17,10,66,1788317,20,26,Avirate Black Dress,Avirate,0.389669
44442,59996,Apparel,Dress,Dresses,Summer,9,1,77,1954794,21,29,Avirate Black & Purple Dress,Avirate,0.426181
44443,59998,Footwear,Shoes,Heels,Winter,16,5,68,2254185,25,34,Catwalk Women Multi Coloured Heels,Catwalk,0.491759
44444,59999,Footwear,Shoes,Heels,Winter,15,13,73,1884894,23,31,Catwalk Women Bronze Wedges,Catwalk,0.410872


## Most Popular Products

In [12]:
df_popular_products = df.sort_values(by="popularity_score", ascending=False).head(10)
df_popular_products

,product_id,masterCategory,subCategory,articleType,season,click_count,item_detail_count,add_to_cart_count,total_dwell_time,homepage_count,purchases,productDisplayName,brands,popularity_score
29968,39631,Footwear,Shoes,Sports Shoes,Summer,27,15,103,4573814,36,38,Nike Men T-Lite White Sports Shoes,Nike,1.0
38577,51921,Accessories,Bags,Handbags,Summer,10,9,86,4514955,23,45,Nyk Women Off White Albedo Handbag,Nyk,0.987075
4817,7261,Footwear,Shoes,Casual Shoes,Fall,33,12,88,4441317,32,47,Rockport Men's Aphrael Black Shoe,Rockport,0.970971
7583,10500,Apparel,Dress,Dresses,Fall,26,12,90,4394518,31,47,Flying Machine Women OS Blue Tops,Flying Machine,0.960724
18902,24493,Accessories,Bags,Backpacks,Winter,27,11,82,4349959,32,60,Wildcraft Unisex Olive Green Backpacks,Wildcraft,0.951001
31968,42865,Accessories,Bags,Handbags,Summer,24,13,87,4245772,29,53,French Connection Women Black Dorris Handbag,French Connection,0.928161
31262,41730,Footwear,Shoes,Heels,Winter,15,19,85,4229407,27,38,Catwalk Women White Heels,Catwalk,0.924496
40412,54812,Accessories,Bags,Handbags,Summer,33,15,92,4224205,25,33,Ivory Tag Grey Fur Handbag,Ivory Tag,0.923358
19374,25149,Accessories,Bags,Handbags,Winter,26,13,74,4209775,23,47,Lino Perros Women Black Handbag,Lino Perros,0.920202
8683,11880,Footwear,Shoes,Formal Shoes,Fall,29,16,72,4182054,27,23,Franco Leone Men Formal Black Formal Shoes,Franco Leone,0.914011


In [17]:
client = bigquery.Client()
BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_PRODUCT_POPULARITY_TABLE_NAME}"
job = client.load_table_from_dataframe(df_popular_products, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))

c:\Users\utkar\.conda\envs\PyCaret\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


## Most Popular Brands

In [19]:
brand_popularity = df.groupby("brands", as_index=False)["popularity_score"].sum()
brand_popularity["popularity_score"] = (brand_popularity["popularity_score"] - brand_popularity["popularity_score"].min()) / (brand_popularity["popularity_score"].max() - brand_popularity["popularity_score"].min())
brand_popularity = brand_popularity.sort_values(by="popularity_score", ascending=False).head(10)
brand_popularity

,brands,popularity_score
336,Nike,1.0
380,Puma,0.879994
3,ADIDAS,0.85042
89,Catwalk,0.603202
490,United Colors of Benetton,0.34852
45,Baggit,0.329952
287,Lino Perros,0.32492
325,Murcia,0.295205
409,Rocia,0.26137
163,Fila,0.238711


In [27]:
client = bigquery.Client()
BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_BRAND_POPULARITY_TABLE_NAME}"
job = client.load_table_from_dataframe(brand_popularity, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))

c:\Users\utkar\.conda\envs\PyCaret\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


## Most Popular SubCategories

In [25]:
subcategory_popularity = df.groupby("subCategory", as_index=False)["popularity_score"].sum()
subcategory_popularity["popularity_score"] = (subcategory_popularity["popularity_score"] - subcategory_popularity["popularity_score"].min()) / (subcategory_popularity["popularity_score"].max() - subcategory_popularity["popularity_score"].min())
subcategory_popularity = subcategory_popularity.sort_values(by="popularity_score", ascending=False).head(10)
subcategory_popularity

,subCategory,popularity_score
30,Shoes,1.0
38,Topwear,0.487475
2,Bags,0.414198
6,Bottomwear,0.085461
42,Watches,0.079049
8,Dress,0.06453
18,Innerwear,0.057404
10,Eyewear,0.034491
19,Jewellery,0.033816
12,Fragrance,0.031836


In [28]:
client = bigquery.Client()
BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_sub_category_POPULARITY_TABLE_NAME}"
job = client.load_table_from_dataframe(subcategory_popularity, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))

c:\Users\utkar\.conda\envs\PyCaret\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


## Top 10 Products for every season and gender

In [31]:
# ============================
# 1️⃣  CONNECT TO BIGQUERY
# ============================

# Initialize BigQuery Client
client = bigquery.Client()

# Define your BigQuery table
bq_table = "data-management-project-452400.data_mgmt_project.click_event_agg"

# Query BigQuery to get the data
query = f"""
SELECT 
    product_id, productDisplayName, gender
FROM `{bq_table}`
"""

# Load Data into Pandas DataFrame
df1 = client.query(query).to_dataframe()

c:\Users\utkar\.conda\envs\PyCaret\lib\site-packages\google\cloud\bigquery\table.py:1900: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [32]:
df1

,product_id,productDisplayName,gender
0,1163,Nike Sahara Team India Fanwear Round Neck Jersey,Men
1,1164,Nike Men Blue T20 Indian Cricket Jersey,Men
2,1165,Nike Mean Team India Cricket Jersey,Men
3,1525,Puma Deck Navy Blue Backpack,Unisex
4,1526,Puma Big Cat Backpack Black,Unisex
...,...,...,...
44441,59995,Avirate Black Dress,Women
44442,59996,Avirate Black & Purple Dress,Women
44443,59998,Catwalk Women Multi Coloured Heels,Women
44444,59999,Catwalk Women Bronze Wedges,Women


In [33]:
df_final = pd.merge(df, df1, left_on="product_id", right_on="product_id", how="outer")

In [34]:
df_final

,product_id,masterCategory,subCategory,articleType,season,click_count,item_detail_count,add_to_cart_count,total_dwell_time,homepage_count,purchases,productDisplayName_x,brands,popularity_score,productDisplayName_y,gender
0,1163,Apparel,Topwear,Tshirts,Summer,0,1,23,331219,12,39,Nike Sahara Team India Fanwear Round Neck Jersey,Nike,0.070414,Nike Sahara Team India Fanwear Round Neck Jersey,Men
1,1164,Apparel,Topwear,Tshirts,Winter,2,3,20,602350,12,29,Nike Men Blue T20 Indian Cricket Jersey,Nike,0.129746,Nike Men Blue T20 Indian Cricket Jersey,Men
2,1165,Apparel,Topwear,Tshirts,Summer,0,0,26,605988,15,31,Nike Mean Team India Cricket Jersey,Nike,0.130573,Nike Mean Team India Cricket Jersey,Men
3,1525,Accessories,Bags,Backpacks,Fall,19,6,73,2884996,23,47,Puma Deck Navy Blue Backpack,Puma,0.630013,Puma Deck Navy Blue Backpack,Unisex
4,1526,Accessories,Bags,Backpacks,Fall,18,14,80,2462089,22,31,Puma Big Cat Backpack Black,Puma,0.537325,Puma Big Cat Backpack Black,Unisex
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44441,59995,Apparel,Dress,Dresses,Fall,17,10,66,1788317,20,26,Avirate Black Dress,Avirate,0.389669,Avirate Black Dress,Women
44442,59996,Apparel,Dress,Dresses,Summer,9,1,77,1954794,21,29,Avirate Black & Purple Dress,Avirate,0.426181,Avirate Black & Purple Dress,Women
44443,59998,Footwear,Shoes,Heels,Winter,16,5,68,2254185,25,34,Catwalk Women Multi Coloured Heels,Catwalk,0.491759,Catwalk Women Multi Coloured Heels,Women
44444,59999,Footwear,Shoes,Heels,Winter,15,13,73,1884894,23,31,Catwalk Women Bronze Wedges,Catwalk,0.410872,Catwalk Women Bronze Wedges,Women


In [35]:
top_10_products_by_season_gender = (
    df_final.groupby(["season", "gender"])
    .apply(lambda x: x.nlargest(10, "popularity_score"))
    .reset_index(drop=True)
)

In [36]:
top_10_products_by_season_gender

,product_id,masterCategory,subCategory,articleType,season,click_count,item_detail_count,add_to_cart_count,total_dwell_time,homepage_count,purchases,productDisplayName_x,brands,popularity_score,productDisplayName_y,gender
0,46884,Footwear,Shoes,Casual Shoes,Fall,20,13,88,3605006,38,54,Ben 10 Boys Black Casual Shoes,Ben 10,0.787816,Ben 10 Boys Black Casual Shoes,Boys
1,12995,Footwear,Shoes,Casual Shoes,Fall,16,4,70,2969015,29,46,Timberland Kids Boys Brown Casual Shoes,Timberland,0.648403,Timberland Kids Boys Brown Casual Shoes,Boys
2,46883,Footwear,Shoes,Casual Shoes,Fall,12,8,60,2933664,19,48,Ben 10 Boys Black & Silver Shoes,Ben 10,0.640633,Ben 10 Boys Black & Silver Shoes,Boys
3,47097,Footwear,Shoes,Casual Shoes,Fall,22,8,78,2723324,22,35,Marvel Boys Red Shoes,Marvel,0.594559,Marvel Boys Red Shoes,Boys
4,47125,Footwear,Shoes,Casual Shoes,Fall,18,8,61,2714430,28,41,Marvel Boys Blue Web Spider Shoes,Marvel,0.592585,Marvel Boys Blue Web Spider Shoes,Boys
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,13174,Footwear,Shoes,Flats,Winter,14,7,78,4010359,22,33,Skechers Women Ever Lasting Black Sandals,Skechers,0.876463,Skechers Women Ever Lasting Black Sandals,Women
185,52339,Accessories,Bags,Clutches,Winter,23,10,76,3967986,28,42,ToniQ Women Black Lace Clutch,ToniQ,0.867223,ToniQ Women Black Lace Clutch,Women
186,19025,Footwear,Shoes,Heels,Winter,23,11,81,3897810,29,66,Catwalk Women Peep Toe Blue Peep Toes,Catwalk,0.851984,Catwalk Women Peep Toe Blue Peep Toes,Women
187,10216,Accessories,Bags,Handbags,Winter,13,13,92,3866941,30,33,Murcia Women Hths Green Handbags,Murcia,0.845099,Murcia Women Hths Green Handbags,Women


In [38]:
client = bigquery.Client()
BQ_PATH = f"{PROJECT_ID}.{BQ_DATASET_NAME}.{BQ_gender_season_POPULARITY_TABLE_NAME}"
job = client.load_table_from_dataframe(top_10_products_by_season_gender, BQ_PATH, job_config=bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE"))

c:\Users\utkar\.conda\envs\PyCaret\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:483: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(
